In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
#from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random 
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# os.chdir(r'D:\senior year\GP\datasets\WordsDataset-thesis')
# if os.path.isdir('train/0/') is False:
#     os.mkdir('train')
#     #os.mkdir('valid')
#     os.mkdir('test')

#     for i in range(0,16): #number of classes
#         shutil.move(f'{i}','train')
# #         os.mkdir(f'valid/{i}')
#         os.mkdir(f'test/{i}')

# #         valid_samples = random.sample(os.listdir(f'train/{i}'),100) #100 sore mnn al train to valid from every class(100*10)
# #         for j in valid_samples:
# #             shutil.move(f'train/{i}/{j}', f'valid/{i}')
# #         print(int((len(f'train/{i}')*20)/100))
#         test_samples = random.sample(os.listdir(f'train/{i}'),15) #50 sora mn al train to test ly kol class
#         for k in test_samples:
#             shutil.move(f'train/{i}/{k}', f'test/{i}')
# os.chdir('../..')


In [3]:
# import cv2 
# import os 
# import glob 
# import numpy as np
# import matplotlib.pyplot as plt 
# from skimage.filters import threshold_otsu

# for i in range(0,10):
#     image_directory = 'D:\senior year\GP\datasets\ragaadataset\AFULL-CLASSES\10 classes with validation\train\{i}'
#     images_path = os.path.join(image_directory, '*g')
#     files = glob.glob(images_path)
#     ostu_images = []
#     for image in files:
#         img = cv2.imread(image)
#         b,g,r = cv2.split(img)
#         rgb_img = cv2.merge([r,g,b])
#         gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#         ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#         #remove noise
#         kernel = np.ones((2,2), np.uint8)
#         closing = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel, iterations = 2)
#         # sure background area
#         sure_bg = cv2.dilate(closing,kernel,iterations=3)
#         # Finding sure foreground area
#         dist_transform = cv2.distanceTransform(sure_bg,cv2.DIST_L2,3)
#         # Threshold
#         ret, sure_fg = cv2.threshold(dist_transform,0.1*dist_transform.max(),255,0)
#         # Finding unknown region
#         sure_fg = np.uint8(sure_fg)
#         unknown = cv2.subtract(sure_bg,sure_fg)
#         # Marker labelling
#         ret, markers = cv2.connectedComponents(sure_fg)
#         # Add one to all labels so that sure background is not 0, but 1
#         markers = markers+1
#         # Now, mark the region of unknown with zero
#         markers[unknown==255] = 0
#         markers = cv2.watershed(img,markers)
#         img[markers == -1] = [255,0,0]
#         ostu_images.append(thresh)
#         plt.imshow(thresh, 'gray')
#         plt.show()

In [4]:
train_path = r'D:\senior year\GP\datasets\WordsDataset-thesis\train'
# valid_path = r'D:\senior year\GP\datasets\ragaadataset\AFULL-CLASSES\10 classes with validation\valid'
test_path = r'D:\senior year\GP\datasets\WordsDataset-thesis\test'

#print('total dataset is 4220 image:')
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, 
    target_size=(224,224),
    batch_size=10)

# valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
#     directory=valid_path,
#     target_size=(224,224), 
#     batch_size=10)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path,
    target_size=(224,224),
    batch_size=10, 
    shuffle=False)

Found 1576 images belonging to 35 classes.
Found 525 images belonging to 35 classes.


In [5]:
mobile= tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [6]:
x = mobile.layers[-6].output #hakhod al hidden layers mn mobile kolo ela akher 6
x

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1024) dtype=float32>

In [7]:
output =  Dense(units=35, activation ='softmax')(x)
#dense new hidden layer

In [8]:
model = Model(inputs=mobile.input, outputs=output) #new model x + dense layer

In [9]:
for layer in model.layers[:-23]: #only train the last 23 layers in our new model
    layer.trainable=False 

In [10]:
model.summary() #our new model (model-6 + dense)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)     

In [11]:
model.compile(optimizer=Adam(lr=0.0100), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import time

start = time.time()
history = model.fit(x=train_batches,
#          steps_per_epoch=len(train_batches), #no batch size, kol epoch btakhod kol al train images
                     batch_size=128,
#          validation_data=valid_batches,
#          validation_steps=len(valid_batches),
         epochs=33,
         verbose=2)
stop = time.time()
print(f"Training time: {(stop - start)/60}min")
#val watya acc 3alya = overfitting

Epoch 1/33
158/158 - 147s - loss: 1.9963 - accuracy: 0.5952
Epoch 2/33


In [ ]:
test_labels = test_batches.classes
test_labels

In [ ]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)


In [ ]:
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))
cm

In [ ]:
def plot_confusion_matrix(cm,classes,
                          normalize=False,
                          title="Confusion Matrix",
                          cmap=plt.cm.Blues):
    #this function prints and plots the confusion matrix
    plt.figure(figsize = (10,10))
    plt.imshow(cm, interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:,np.newaxis]
        print("normalized confusion matrix")
    else:
        print('confusion matrix, without normalization')
        
    print(cm)
    
    thresh = cm.max()/2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], 
                horizontalalignment = "center",
                color="white" if cm[i,j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('true/real label')
    plt.xlabel('predicted label')

In [ ]:
test_batches.class_indices

In [ ]:
number_of_classes = 35
cm_plot_labels = []
for i in range(0, number_of_classes):
    cm_plot_labels.append(i)
    
print(cm_plot_labels)    

In [ ]:
# cm_plot_labels=['0','1','2','3','4','5','6','7','8','9']
# cm_plot_labels=['0','1','2','3','4']
# cm_plot_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105]

plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title="conf matrix")

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
# test_x=np.array(test_samples)
# y_pred=model.predict(test_x)
# y_pred=np.argmax(y_pred,axis=1)

y_pred=predictions.argmax(axis=1)
print(classification_report(y_pred,test_batches.classes))

In [ ]:
plt.plot(history.history['accuracy'], label='train acc')
# plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
model.save(r"D:\senior year\GP\draft code\h5 models\thesis\Words35-thesis.h5")

In [ ]:
# import Augmentor

# # Build a new pipeline for image processing.
# p = Augmentor.Pipeline(
#     source_directory="D:/senior year/GP/datasets/WordsDataset-thesis/15",
#     output_directory="D:/senior year/GP/datasets/WordsDataset-thesis/15")

# # Manipulate the example card.
# p.rotate_without_crop(
#     probability=.5,
#     max_left_rotation=10,
#     max_right_rotation=10,
#     expand=True)
# # p.zoom(
# #     probability=.3,
# #     min_factor=.8,
# #     max_factor=1.1)
# p.skew(
#     probability=.3,
#     magnitude=.15)
# p.random_brightness(
#     probability=.5,
#     min_factor=.5,
#     max_factor=1.75)
# p.flip_left_right(probability=0.4)

# # Generate and save X new images to the output directory.
# p.sample(57)